In [1]:
import pandas as pd
import random

In [2]:
path = "../interim/interim_1/tech_test.txt"

with open(path,'r') as reader:
    data = reader.read().strip().split('\n\n')

In [3]:
data[:2]

['Gab S-ORG\nappears O\nto O\nbe O\nlosing O\ninvestors O\nafter O\nPittsburgh S-LOC\nshooting O',
 'Gab S-ORG\n, O\na O\nsocial O\nmedia O\nplatform O\npopular O\nwith O\nfar-right O\nextremists O\n, O\nappears O\nto O\nhave O\nlost O\nmore O\nthan O\nthree O\ndozen O\nsmall O\ninvestors O\nin O\nthe O\nfour O\ndays O\nsince O\na O\nman O\nposted O\non O\nthe O\nsite O\n, O\n" O\nScrew O\nyour O\noptics O\n, O\nI O\n\'m O\ngoing O\nin O\n, O\n" O\nand O\nthen O\nallegedlyburst O\ninto O\na O\nPittsburgh S-LOC\nsynagogue O\nand O\nopened O\nfire O\n, O\nkilling O\n11 O\npeople O\nand O\ninjuring O\nsix O\nothers O\n. O']

In [4]:
for i in range(len(data)):
    line = data[i].strip()
    splitted_line = line.split('\n')
    result_line = []
    for j,token_bio_tag in enumerate(splitted_line):
        try:
            token, bio_tag = token_bio_tag.split()
        except Exception as e:
            print(token_bio_tag)
            print(line)
            print(j)
            print(splitted_line[24:26])
            raise e
        bio_tag = bio_tag.replace("S-","B-")
        bio_tag = bio_tag.replace("E-","I-")
        result_line.append(f"{bio_tag}\t{token}")
    result_line = '\n'.join(result_line)
    data[i] = result_line

In [5]:
data[:3]

['B-ORG\tGab\nO\tappears\nO\tto\nO\tbe\nO\tlosing\nO\tinvestors\nO\tafter\nB-LOC\tPittsburgh\nO\tshooting',
 'B-ORG\tGab\nO\t,\nO\ta\nO\tsocial\nO\tmedia\nO\tplatform\nO\tpopular\nO\twith\nO\tfar-right\nO\textremists\nO\t,\nO\tappears\nO\tto\nO\thave\nO\tlost\nO\tmore\nO\tthan\nO\tthree\nO\tdozen\nO\tsmall\nO\tinvestors\nO\tin\nO\tthe\nO\tfour\nO\tdays\nO\tsince\nO\ta\nO\tman\nO\tposted\nO\ton\nO\tthe\nO\tsite\nO\t,\nO\t"\nO\tScrew\nO\tyour\nO\toptics\nO\t,\nO\tI\nO\t\'m\nO\tgoing\nO\tin\nO\t,\nO\t"\nO\tand\nO\tthen\nO\tallegedlyburst\nO\tinto\nO\ta\nB-LOC\tPittsburgh\nO\tsynagogue\nO\tand\nO\topened\nO\tfire\nO\t,\nO\tkilling\nO\t11\nO\tpeople\nO\tand\nO\tinjuring\nO\tsix\nO\tothers\nO\t.',
 'O\tIn\nO\tthe\nO\tdays\nO\tsince\nB-ORG\tGab\nO\trose\nO\tto\nO\tnational\nO\tprominence\nO\tbecause\nO\tof\nO\tthe\nO\tviolent\nO\tand\nB-MISC\tanti-Semitic\nO\tpostings\nO\tmade\nO\tby\nO\taccused\nO\tshooter\nB-PER\tRobert\nI-PER\tBowers\nO\t,\nO\tthe\nO\tsite\nO\twas\nO\ttaken\nO\tdown\nO\t,\

In [6]:
def preprocess_line(line):
    splitted_line = line.split('\n')
    text = []
    named_entity = {}
    prev_bio_tag = None
    current_named_entity = []
    for bio_tag_token in splitted_line:
        bio_tag, token = bio_tag_token.split('\t')
        text.append(token)

        # 1. if bio tag is B
        #   1.1. prev tag is None
        #   1.2. prev tag is O
        #   1.3. prev tag is B
        #       1.3.1. prev tag label is same
        #       1.3.2. prev tag label is different
        #   1.4. prev tag is I
        #       1.4.1. prev tag label is same
        #       1.4.2. prev tag label is different
        # 2. if bio tag is I
        #   2.1. prev tag is None
        #   2.2. prev tag is O
        #   2.3. prev tag is B
        #       2.3.1. prev tag label is same
        #       2.3.2. prev tag label is different
        #   2.4. prev tag is I
        #       2.4.1. prev tag label is same
        #       2.4.2. prev tag label is different
        # 3. if bio tag is O
        #   3.1. prev tag is None
        #   3.2. prev tag is O
        #   3.3. prev tag is B
        #   3.4. prev tag is I
        if bio_tag.startswith("B-"):
            if prev_bio_tag == None:
                # Append token
                current_named_entity.append(token)
            elif prev_bio_tag == 'O':
                # Append token
                current_named_entity.append(token)
            elif prev_bio_tag.startswith("B-"):
                tag_label = bio_tag[2:]
                prev_tag_label = prev_bio_tag[2:]
                if tag_label == prev_tag_label:
                    if not prev_tag_label in named_entity.keys():
                        named_entity[prev_tag_label] = []
                    named_entity[prev_tag_label].append(' '.join(current_named_entity))
                    current_named_entity = [token]
                else:
                    if not prev_tag_label in named_entity.keys():
                        named_entity[prev_tag_label] = []
                    named_entity[prev_tag_label].append(' '.join(current_named_entity))
                    current_named_entity = [token]
            elif prev_bio_tag.startswith("I-"):
                tag_label = bio_tag[2:]
                prev_tag_label = prev_bio_tag[2:]
                if tag_label == prev_tag_label:
                    if not prev_tag_label in named_entity.keys():
                        named_entity[prev_tag_label] = []
                    named_entity[prev_tag_label].append(' '.join(current_named_entity))
                    current_named_entity = [token]
                else:
                    if not prev_tag_label in named_entity.keys():
                        named_entity[prev_tag_label] = []
                    named_entity[prev_tag_label].append(' '.join(current_named_entity))
                    current_named_entity = [token]
        elif bio_tag.startswith("I-"):
            if prev_bio_tag == None:
                raise Exception(f"I token cannot begin a named entity phrase | line : {line}")
            elif prev_bio_tag == 'O':
                raise Exception(f"I token cannot begin a named entity phrase | line : {line}")
            elif prev_bio_tag.startswith("B-"):
                tag_label = bio_tag[2:]
                prev_tag_label = prev_bio_tag[2:]
                if tag_label == prev_tag_label:
                    # Append token
                    current_named_entity.append(token)
                else:
                    raise Exception(f"I token cannot align with B token with different label | line : {line}")
            elif prev_bio_tag.startswith("I-"):
                tag_label = bio_tag[2:]
                prev_tag_label = prev_bio_tag[2:]
                if tag_label == prev_tag_label:
                    # Append token
                    current_named_entity.append(token)
                else:
                    raise Exception(f"I token cannot align with I token with different label | line : {line}")
        elif bio_tag == 'O':
            if prev_bio_tag == None:
                pass
            elif prev_bio_tag == 'O':
                pass
            elif prev_bio_tag.startswith("B-"):
                prev_tag_label = prev_bio_tag[2:]
                if not prev_tag_label in named_entity.keys():
                    named_entity[prev_tag_label] = []
                named_entity[prev_tag_label].append(' '.join(current_named_entity))
                current_named_entity = []
            elif prev_bio_tag.startswith("I-"):
                prev_tag_label = prev_bio_tag[2:]
                if not prev_tag_label in named_entity.keys():
                    named_entity[prev_tag_label] = []
                named_entity[prev_tag_label].append(' '.join(current_named_entity))
                current_named_entity = []
        else:
            raise Exception(f"Unknown tag {bio_tag}")
        prev_bio_tag = bio_tag
    
    if prev_bio_tag != 'O': # last token
        prev_tag_label = prev_bio_tag[2:]
        if not prev_tag_label in named_entity.keys():
            named_entity[prev_tag_label] = []
        named_entity[prev_tag_label].append(' '.join(current_named_entity))
        current_named_entity = []
    
    return ' '.join(text), named_entity

In [7]:
data = [preprocess_line(line) for line in data]

In [8]:
data[:5]

[('Gab appears to be losing investors after Pittsburgh shooting',
  {'ORG': ['Gab'], 'LOC': ['Pittsburgh']}),
 ('Gab , a social media platform popular with far-right extremists , appears to have lost more than three dozen small investors in the four days since a man posted on the site , " Screw your optics , I \'m going in , " and then allegedlyburst into a Pittsburgh synagogue and opened fire , killing 11 people and injuring six others .',
  {'ORG': ['Gab'], 'LOC': ['Pittsburgh']}),
 ('In the days since Gab rose to national prominence because of the violent and anti-Semitic postings made by accused shooter Robert Bowers , the site was taken down , and a count of its investors on a crowdfunding page has slowly declined .',
  {'ORG': ['Gab'], 'MISC': ['anti-Semitic'], 'PER': ['Robert Bowers']}),
 ('On the website StartEngine.com , Gab is soliciting investments of at least $ 252 for " $ 4/share of Class B Non-Voting Common Stock deliverable in token form ( the \' GAB Tokens \' ) . "',
  

In [9]:
value_counts = {}
for el in data:
    text, named_entity = el
    for k,v in named_entity.items():
        if k not in value_counts.keys():
            value_counts[k] = 0
        value_counts[k] += len(v)

In [10]:
value_counts

{'ORG': 875, 'LOC': 489, 'MISC': 365, 'PER': 1094}

In [11]:
entity_map = {
    "ORG" : "Organization",
    "LOC" : "Location",
    "MISC" : "Miscellaneous",
    "PER" : "Person"
}

In [12]:
mask = "<extra_id_X>"
for i in range(len(data)):
    inputs = f"Extract NER with format >> entity : <extra_id_0>, entity_type : <extra_id_1> | {data[i][0]}"
    if len(data[i][1]) > 0:
        out = []
        cnt = 0
        for k,v in data[i][1].items():
            for v2 in v:
                cnt = cnt%100
                tup = f"{mask.replace('X',str(cnt))} {v2} {mask.replace('X',str(cnt+1))} {entity_map[k].lower()}"
                cnt += 2
                out.append(tup)
        out = " ; ".join(out)
    else:
        out = "NULL"
    data[i] = {
        "input" : inputs,
        "output" : out
    }

In [13]:
data[0]

{'input': 'Extract NER with format >> entity : <extra_id_0>, entity_type : <extra_id_1> | Gab appears to be losing investors after Pittsburgh shooting',
 'output': '<extra_id_0> Gab <extra_id_1> organization ; <extra_id_2> Pittsburgh <extra_id_3> location'}

In [14]:
import pandas as pd

df = pd.DataFrame(data)

In [15]:
df.to_csv("../interim/interim_2/tech.csv", index=False)